In [ ]:
!pip install '/kaggle/input/competitionpackages/wheels/torch-1.7.1+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/competitionpackages/wheels/torchvision-0.8.2+cu110-cp37-cp37m-linux_x86_64.whl' --no-deps
!pip install '/kaggle/input/competitionpackages/wheels/torchaudio-0.7.2-cp37-cp37m-manylinux1_x86_64.whl' --no-deps

In [ ]:
!pip install '/kaggle/input/1145141919810/mmdetectionv2180/addict-2.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/1145141919810/mmdetectionv2180/yapf-0.31.0-py2.py3-none-any.whl' --no-deps
!pip install '/kaggle/input/1145141919810/mmdetectionv2180/terminal-0.4.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/1145141919810/mmdetectionv2180/terminaltables-3.1.0-py3-none-any.whl' --no-deps
!pip install '/kaggle/input/1145141919810/mmdetectionv2180/mmcv_full-1_3_17-cu110-torch1_7_1/mmcv_full-1.3.17-cp37-cp37m-manylinux1_x86_64.whl' --no-deps
!pip install '/kaggle/input/1145141919810/mmdetectionv2180/pycocotools-2.0.2/pycocotools-2.0.2' --no-deps
!pip install '/kaggle/input/1145141919810/mmdetectionv2180/mmpycocotools-12.0.3/mmpycocotools-12.0.3' --no-deps

In [ ]:
!rm -rf mmdetection

!cp -r /kaggle/input/1145141919810/mmdetectionv2180/mmdetection-2.18.0 /kaggle/working/
!mv /kaggle/working/mmdetection-2.18.0 /kaggle/working/mmdetection
%cd /kaggle/working/mmdetection
!pip install -e .
#!rm /kaggle/working/mmdetection/configs/_base_/datasets/coco_instance.py
#!mv /kaggle/input/cocoinstance/coco_instance.py /kaggle/working/mmdetection/configs/_base_/datasets

In [ ]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
import torch.nn.functional as F
import sklearn
import torchvision
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
import cupy as cp
import gc
import pandas as pd
import os
import matplotlib.pyplot as plt
import PIL
import json
from PIL import Image, ImageEnhance
import albumentations as A
import mmdet
import mmcv
from albumentations.pytorch import ToTensorV2
import seaborn as sns
import glob
from pathlib import Path
import pycocotools
from pycocotools import mask
import numpy.random
import random
import cv2
import re
import shutil
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.apis import inference_detector, init_detector, show_result_pyplot, set_random_seed

In [ ]:
%cd ..

In [ ]:
IMG_WIDTH = 704
IMG_HEIGHT = 520

In [ ]:
def rle_decode(mask_rle, shape):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height,width) of array to return 
    Returns numpy array, 1 - mask, 0 - background

    '''
    s = mask_rle.split()
    starts, lengths = [np.asarray(x, dtype=int) for x in (s[0:][::2], s[1:][::2])]
    starts -= 1
    ends = starts + lengths
    img = np.zeros(shape[0]*shape[1], dtype=np.uint8)
    for lo, hi in zip(starts, ends):
        img[lo:hi] = 1
    return img.reshape(shape)

def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    return ' '.join(str(x) for x in runs)

In [ ]:
def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))

In [ ]:
def get_mask_from_result(result):
    d = {True : 1, False : 0}
    u,inv = np.unique(result,return_inverse = True)
    mk = cp.array([d[x] for x in u])[inv].reshape(result.shape)
#     print(mk.shape)
    return mk

In [ ]:
def does_overlap(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            #import pdb; pdb.set_trace()
            #print("Found overlapping masks!")
            return True
    return False


def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            print("Overlap detected")
            mask[np.logical_and(mask, other_mask)] = 0
    return mask

**model**

In [ ]:
# %%writefile labels.txt
# shsy5y
# cort
# astro

In [ ]:
from mmcv import Config
cfg = Config.fromfile('/kaggle/working/mmdetection/configs/cascade_rcnn/cascade_mask_rcnn_x101_64x4d_fpn_20e_coco.py')

In [ ]:
cfg.dataset_type = 'CocoDataset'
cfg.classes = '/kaggle/working/labels.txt'
cfg.data_root = '/kaggle/working'

for head in cfg.model.roi_head.bbox_head:
    head.num_classes = 3
    
cfg.model.roi_head.mask_head.num_classes=3

cfg.data.test.type = 'CocoDataset'
cfg.data.test.classes = 'labels.txt'
cfg.data.test.data_root = '../input/livecellshsy5y'
cfg.data.test.ann_file = 'annotations_val.json'
cfg.data.test.img_prefix = '/kaggle/input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/images/livecell_train_val_images/SHSY5Y/'

cfg.data.train.type = 'CocoDataset'
cfg.data.train.data_root = '../input/livecellshsy5y'
cfg.data.train.ann_file = 'annotations_train.json'
cfg.data.train.img_prefix = '/kaggle/input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/images/livecell_train_val_images/SHSY5Y/'
cfg.data.train.classes = 'labels.txt'

cfg.data.val.type = 'CocoDataset'
cfg.data.val.data_root = '../input/livecellshsy5y'
cfg.data.val.ann_file = 'annotations_val.json'
cfg.data.val.img_prefix = '/kaggle/input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/images/livecell_train_val_images/SHSY5Y/'
cfg.data.val.classes = 'labels.txt'

cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='RandomFlip', flip_ratio=0.5),
    dict(type='Resize', img_scale=[(1333, 800), (2666, 1600)], keep_ratio=True),
    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'), 
    dict(type='Collect', keys=['img', 'gt_masks'])
]

cfg.val_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations', with_bbox=True, with_mask=True),
    dict(type='RandomFlip', flip_ratio=0.5),

    dict(
        type='Normalize',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'), 
    dict(type='Collect', keys=['img', 'gt_masks'])
    #""", 'gt_bboxes', 'gt_masks', 'gt_labels']""")
]

cfg.test_pipeline = [
    dict(type='LoadImageFromWebcam'),
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations',with_mask=True),
    dict(type='Resize', img_scale=[(480, 1333), (520, 1333),
                                   (640, 1333), (704, 1333),
                                   (768, 1333)], multiscale_mode='value', keep_ratio=True),
    dict(
        type='Normalize',
        mean=[128, 128, 128],
        std=[11.58, 11.58, 11.58],
        to_rgb=True),
    dict(type='Pad', size_divisor=32),
    dict(type='DefaultFormatBundle'),
    dict(type='Collect', keys=['img'])
]

cfg.work_dir = '/kaggle/working/model_output/cascade_mask_rcnn'

cfg.data.samples_per_gpu = 2
cfg.data.workers_per_gpu = 2

cfg.data.train.pipeline = cfg.train_pipeline
cfg.model.test_cfg.rcnn.max_per_img = 540
#cfg.data.test.pipeline = cfg.test_pipeline
#cfg.data.test.pipeline[0] = dict(type = 'LoadImageFromWebcam')

cfg.evaluation.metric = 'segm'
cfg.evaluation.interval = 4

cfg.checkpoint_config.interval = 4
cfg.runner.max_epochs = 12
cfg.log_config.interval = 50

cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
# cfg.fp16 = dict(loss_scale=512.0)
# meta = dict()
# meta['config'] = cfg.pretty_text
print(f'Config:\n{cfg.pretty_text}')

**inference**

In [ ]:
masks = []
files = []

In [ ]:
confidence_thresholds = {0: 0.15, 1: 0.55, 2: 0.35}

In [ ]:
# datasets = [build_dataset(cfg.data.train)]
model = init_detector(cfg, '../input/epoch-9/sys5h_epoch6.pth')

for file in sorted(os.listdir('../input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/images/livecell_train_val_images/SHSY5Y/')):
    img = mmcv.imread('../input/sartorius-cell-instance-segmentation/LIVECell_dataset_2021/images/livecell_train_val_images/SHSY5Y/' + file)
    result = inference_detector(model, img)
    show_result_pyplot(model, img, result)
    msk = []
    for i, classe in enumerate(result[0]):
        if classe.shape != (0, 5):
            bbs = classe
#             print(bbs)
            sgs = result[1][i]
            for bb, sg in zip(bbs,sgs):
                box = bb[:4]
                cnf = bb[4]
                if cnf >= confidence_thresholds[i]:
                    mask = get_mask_from_result(sg)
                    mask = remove_overlapping_pixels(mask, msk)
                    msk.append(mask)
                
    for mk in msk:
            rle_mask = rle_encoding(mk)
            masks.append(rle_mask)
            files.append(str(file.split('.')[0]))

In [ ]:
files = pd.Series(files, name='id')
preds = pd.Series(masks, name='predicted')

In [ ]:
submission_df = pd.concat([files, preds], axis=1)

In [ ]:
submission_df.to_csv('submission.csv', index=False)

In [ ]:
submission_df

In [ ]:
shutil.rmtree('/kaggle/working/mmdetection')